# Basic statistics

This notebook is a collation of basic stats study materials from a few sources.
The aim is to create a single notebook where I can record my learnings and quickly look up concepts.

Sources:
- [A Cartoon Guide to statistics](https://www.amazon.es/Cartoon-Guide-Statistics/dp/0062731025)
- [Hackerrank - 10 days of statistics](https://www.hackerrank.com/domains/tutorials/10-days-of-statistics)
- [Rice University - Online Statistics Education](https://onlinestatbook.com/2/index.html)
- [Ace the Data Science Interview](https://www.amazon.com/Ace-Data-Science-Interview-Questions/dp/0578973839)
- [Machine learning mastery](https://machinelearningmastery.com)

Contents:
- Data description
- Probability
- Random variables
- Distributions
- Sampling
- Confidence intervals
- Hypothesis testing
- Comparing two populations
- Regression
- Correlations

Not covered in above (so far)
- Bayes likelihood
- Confusion matrix / ROC / AUC
- Multiple testing correction (product book)

In [ ]:
# Use latest code in directory. Alternatively, run poetry install so the package is used instead.
import os
os.chdir("..")

In [ ]:
from collections import defaultdict
import logging
import math
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

# from ndj_pipeline import db, model, transform, utils

logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s [%(levelname)s] %(message)s", 
    handlers=[logging.StreamHandler()]
)

In [ ]:
input_path = Path("data", "processed", "titanic.parquet")
logging.info(f"Loading data from {input_path}")
data = pd.read_parquet(input_path)

data = data.dropna(subset=['age', 'sex'])
data['name_u'] = data['name'].str.contains('u').astype(int)

# Data description

- Summary statistics
    - x1, x2, x3, xn
    - Mean x̄
    - Σ summation
    - Median
    - Interquartile range
- Variability and Standard deviation (s)
- Z scores

In [ ]:
# Mean, median, mode
# https://www.hackerrank.com/challenges/s10-basic-statistics/problem?isFullScreen=true
x_raw = "64630 11735 14216 99233 14470 4978 73429 38120 51135 67060"

x = [int(x) for x in x_raw.split(" ")]
x = sorted(x)

# Mean
mean = sum(x) / len(x)

# Mode
if len(x) % 2 == 0:
    half = int(len(x) / 2)
    median = x[half-1:half+1]
    median = sum(median) / 2
else:
    half = int(len(x) / 2)
    median = x[half]
    
# Mode
counts = defaultdict(int)
for num in x:
    counts[num] += 1
    
max_count = max(counts.values())
mode = min([num for num, count in counts.items() if count==max_count])

print(f"{mean:.1f}")
print(f"{median:.1f}")
print(f"{mode}")

In [ ]:
# Weighted mean
def weightedMean(X, W):
    sum_product = 0
    for i in range(len(W)):
        sum_product += X[i] * W[i]

    answer = sum_product / sum(W)
    print(f"{answer:.1f}")
    
    
# n = int(input().strip())
# vals = list(map(int, input().rstrip().split()))
# weights = list(map(int, input().rstrip().split()))
# weightedMean(vals, weights)

In [ ]:
# Quartiles and interquartile range
def quartiles(arr):
    arr = sorted(arr)

    mid = get_lower_mid_index(arr)    
    q2 = get_median(arr)

    if len(arr) % 2:
        q1 = arr[:mid]
        q3 = arr[mid + 1:]
    else:
        q1 = arr[:mid + 1]
        q3 = arr[mid + 1:]
        
    q1 = get_median(q1)
    q3 = get_median(q3)
        
    return [q1, q2, q3]


def get_median(arr):
    mid = get_lower_mid_index(arr)
    if len(arr) % 2:
        median = arr[mid]
    else:
        median = sum(arr[mid:mid + 2]) / 2
    return median


def get_lower_mid_index(arr):
    return int((len(arr) - 1) / 2)

    
def interQuartile(values, freqs):
    all_values = get_all_values(values, freqs)
    q1, q2, q3 = quartiles(all_values)
    print(round(float(q3) - float(q1), 1))
    

def get_all_values(values, freqs):
    all_values = []
    for i in range(len(freqs)):
        for f in range(freqs[i]):
            all_values.append(values[i])
    return all_values

In [ ]:
x_raw = "64630 11735 14216 99233 14470 4978 73429 38120 51135 67060"
x = [int(x) for x in x_raw.split(" ")]
quartiles(x)

In [ ]:
# Variation and standard deviation
def stdDev(X):    
    mean = sum(X) / len(X)

    variance = sum([(mean - x)**2 for x in X]) / len(X)
    std = variance**0.5
    print(f"{std:.1f}")

In [ ]:
print(x)
stdDev(x)

# Probability

- Basic definitions
    - Random experiment
    - Elementary outcomes
    - Sample space
    - P(Oi) >= 0
    - P(O1) + P(O2) + P(On) = 1
- Event, event elementary outcomes, probability
- Addition rule
    - P(E OR F) = P(E) + P(F) - P(E AND F)
    - P(E OR F) = P(E) + P(F)  (mutually exclusive only)
- Subtraction rule
    - P(E) = 1 - P(NOT E)
    - Very useful for ‘chance no 12’s are thrown’ problems
- Conditional probability
    - P(E | F) = P(E and F) / P(F)
- Multiplication rule
    - P(E and F) = P(E | F) * P(F)
    - P(E AND F) = P(E) * P(F) (if independent)
- Disease example
    - Setup
        - 1 / 1000 infected
        - Positive 99% of the time for those with disease
        - 2% of uninfected patients also test positive
        - You tested positive, what is your chance of having disease?
    - Chance of having disease goes to 0.0472 if positive

In [ ]:
# What is the chance I receive at least one offer from a company?
# Chance of an offer
google = 0.05
facebook = 0.6
lyft = 0.333

1 - ((1 - google) * (1 - facebook) * (1 - lyft))

In [ ]:
# What is the probability that I get an offer from all companies?
google * facebook * lyft

In [ ]:
# Two people throw a biased coin with 60% of heads
# Conditional probability: Whats the probability of both heads coming up? 
# What about after knowing 'someone' has heads?
print(0.36 / (0.24 + 0.24 + 0.36))

pd.DataFrame([[0.4*0.4, 0.6*0.4], [0.4*0.6, 0.6*0.6]])

In [ ]:
# Example disease problem
# Rare disease affects 1 in 1000 people
# If person has disease, test comes back positive 99% of time
# If person does not have disease, test comes back positive 2% of time
# What are chances of having the disease if you test positive?


print(99 / (99+1998))
pd.DataFrame([[97902, 1], [1998, 99]])

# Method: Get a large number of people as the table total, i.e. 100k
# Fill in the disease column, then 2% false positive, then leftover

In [ ]:
# Birthday problem, given n people, what are odds at least two people share a birthday?
n = 23

prob = 1
for i in range(n):
    available_days = 365 - i
    prob *= available_days / 365
    
1 - prob

In [ ]:
# Monty hall
# Three doors, there is a car behind one, you pick door A.
# Monty removes a door which does not have a car i.e. door C.
# Should you stay or switch? What are the odds?
doors = 3
remove = 1

options = doors - remove - 1

correct_first_time = 1 / doors
correct_after_switch = (1 - correct_first_time) / options

print(correct_first_time, correct_after_switch)

# Random variables

- Random variable 
    - X
    - Numerical outcome of a random experiment
    - Features of a data point
    - x is a single value of X, i.e. category levels
- Probability distributions (model or population properties)
    - Mean of X - μ
    - Variance of X - σ
- Expected value
    - E[X]
    - μ = Σ x * p(x)
- Expected variance
    - σ² = Σ(x - μ)² * p(x)
- Continuous random variables
    - A specific point has probability = 0 (continuous)
    - Probability density function (+ cumulative)
    - μ =∫ x* f(x)dx
    - σ² = ∫ (x - μ)² * f(x)dx
- Adding random variables
    - E[aX +b] = a * E[X] + b
    - σ²(aX + b) = a²σ²(X)
    - Many random variables
        - Expected: Σ E[Xi]
        - Variance: Σσ²(Xi)

A large elevator can transport a maximum of  pounds. Suppose a load of cargo containing  boxes must be transported via the elevator. The box weight of this type of cargo follows a distribution with a mean of  pounds and a standard deviation of  pounds. Based on this information, what is the probability that all  boxes can be safely loaded into the freight elevator and transported?

In [ ]:
max_weight = 9800
number_boxes = 49
box_mean = 205
box_sd = 15

total_mean = number_boxes * box_mean
total_sd = box_sd * number_boxes ** 0.5

# Probability all can be loaded into elevator
stats.norm.cdf(max_weight, total_mean, total_sd)

# A tale of two distributions

- Bernoulli trials
- Binomial random variable
    - x successes in n repeated trials with p success probability
    - (n choose k) * p ** k * (1 - p) ** (n - k)
    - f = math.factorial
    - f(a) / (f(b) * f(a - b))
    - Pascal’s triangle
    - μ = np
    - σ² = np(1-p)
- Poisson distribution
    - p = ((math.e ** -u) * (u ** x)) / x!
- Standard normal distribution
    - (1 / (2 * pi)**0.5) * e ^ - (z² / 2)
    - μ = 0
    - σ = 1
    - z = x - μ / σ
- Fuzzy central limit theorem
    - Data that are influenced by many small and unrelated random effects are approximately normally distributed
    - SN distribution approximates binomial random variables 

In [ ]:
# Combinations can be used to form Pascal's triangle
for i in range(10+1):
    print(math.comb(10,i))

In [ ]:
# How many failures over x bernulli trials?
# Nail factory gives us 10 samples, nails are usually good (88% of time).
p_ok = 0.88
p_fail = 1 - p_ok

# Will sum to 1
(  (p_ok ** 10 * p_fail**0) * 1 # no failures
 + (p_ok ** 9 * p_fail**1) * 10 # one failure
 + (p_ok ** 8 * p_fail**2) * 45 # two failures
#  + (p_ok ** 7 * p_fail**3) * 120
#  + (p_ok ** 6 * p_fail**4) * 210
#  + (p_ok ** 5 * p_fail**5) * 252
#  + (p_ok ** 4 * p_fail**6) * 210
#  + (p_ok ** 3 * p_fail**7) * 120
#  + (p_ok ** 2 * p_fail**8) * 45 
#  + (p_ok ** 1 * p_fail**9) * 10
#  + (p_ok ** 0 * p_fail**10) * 1 
)

In [ ]:
# Poisson distribution

Acme Realty company sells an average of 2 homes per day. What is the probability that exactly 3 homes will be sold tomorrow?

In [ ]:
lm = 2
k = 3

(lm**k * math.e**(-lm))  / math.factorial(k)

Suppose the average number of lions seen by tourists on a one-day safari is 5. What is the probability that tourists will see fewer than 3 lions on the next one-day safari?

In [ ]:
lm = 5
k = 3

(
(lm**0 * math.e**(-lm))  / math.factorial(0)
+ (lm**1 * math.e**(-lm))  / math.factorial(1)
+ (lm**2 * math.e**(-lm))  / math.factorial(2)
+ (lm**3 * math.e**(-lm))  / math.factorial(3)
)

In [ ]:
# DnD chance of getting 2x 20's
f = math.factorial

# success is odds of getting an 18 with 4 dice drop 1
# on a single trial
s = 6*4 / 6**4
n = 6
k = 2

# over 6 rolls, what are chances of getting 2?
n_choose_k = int(f(n) / (f(k) * f(n - k)))
prob = (n_choose_k) * s**k * (1 - s)**(n - k)
prob = round(prob, 5)

print(f"success: {round(s*100, 2)}%, ways: {n_choose_k} of {f(n)}, prob: {round(prob*100,2)}%")

In [ ]:
# Probability of getting at least one six in four rolls
1 - (5 / 6)**4

In [ ]:
# Probability of getting one double six in 24 rolls
1 - (35 / 36) ** 24

### Normal distribution
![normal_distribution](http://mathbitsnotebook.com/Algebra2/Statistics/normalstandard.jpg)

# Sampling

- Everything governed by 1 / n ** 0.5
- Sample size and standard error
    - Don’t know population p, can estimate p̂ from sample
    - p̂ = x / n
    - p̂ can be considered a random variable (Capital p̂)
    - SD: o(Capital p̂) = (p * (1 - p)) ** 0.5 / n ** 0.5
    - Thus spread governed by 1 / n ** 0.5
- Sampling error
    - From above, SD of P hat
    - Increasing sample size by 4 reduces spread by 2
- Sampling distribution of the mean
    - Sample mean x̄ = (x1 + x2 + xn) / n
    - E[x̄] = μ
    - σ(x̄) = σ / n ** 0.5 
    - Variances of Xi/n add to give variance of x̄
- Central limit theorem
    - Random samples of size n
    - As n gets large, x̄ approaches normal distribution
    - With mean μ and SD σ / n ** 0.5
- Student’s t-distribution
    - CLT depends on large sample size and need known SD
    - Use unbiased standard deviation of the sample
    - 1 / (n - 1) Σ (xi - x̄)² 
    - Introduces more uncertainty through 1-n

In [ ]:
# CDF
# norm.cdf: cumulative distribution function is sample, mean, SD. 
# Or use it with the Z score

x = stats.norm.cdf(19.5, 20, 2)
logging.info(f"-0.25 SD cumulative: {x:.3f}")

x = stats.norm.cdf(-0.25)
logging.info(f"Same same: {x:.3f}")

x = stats.norm.cdf(22, 20, 2) - stats.norm.cdf(20, 20, 2)
logging.info(f"1 SD difference: {x:.3f}")

# Confidence intervals

- Opposite of the above, given a sample what can infer for pop
- Proportion confidence intervals (standard error)
    - SE(p̂) = (p̂ * (1 - p̂)) ** 0.5 / n ** 0.5
- How much sample n given confidence and acceptable error
    - estimated_n = (z_score ** 2 * p * (1 - p)) / acceptable_error ** 2
- Means confidence intervals (standard error)
    - sample_error = ssd / n ** 0.5
    - 95% confidence is sample_error * z score (1.96)
- Student’s T
    - Useful for smaller samples i.e. less than 50
- All standard errors are proportional to 1 / n ** 0.5

In [ ]:
# Polling example; How much sample is required to have small error with high confidence?
confidence = 0.99
error = 0.01
p = 0.5 # a guess

# One tailed
z = stats.norm.ppf(confidence + (1 - confidence) / 2)

n = (z**2 * (p * (1-p))) / (
error ** 2
)
print(f"z score: {z:.3f}, sample needed: {n:.0f}")

In [ ]:
# Confidence interval for a proportion
p̂ = 0.55
n = 1000
SD_p̂ = (p̂ * (1 - p̂)) **0.5 / n **0.5
SD_p̂

In [ ]:
# Thus, with 95% confidence (z score), population probability is within 0.55 +- 0.031
# Can use different confidence levels using different z scores
SD_p̂ * -1.96, SD_p̂ * 1.96

In [ ]:
# NOTE; the above IS VERY SIMILAR to 1/sqrt(n)
1 / (1000**0.5)

In [ ]:
# The closer p̂ is to 50%, the more uncertainty, closer to 1 or 0, less uncertainty
p̂ = 0.9
n = 1000
SD_p̂ = (p̂ * (1 - p̂)) **0.5 / n **0.5
SD_p̂

In [ ]:
# The higher n, the less uncertainty; remember 4x n is about 2x less uncertainty
p̂ = 0.5
n = 4000
SD_p̂ = (p̂ * (1 - p̂)) **0.5 / n **0.5
SD_p̂

In [ ]:
# If we want 99% confidence with an error +- 0.01, what n?
z = 2.58
p = 0.5
e = 0.01

n = (z**2 * p * (1 - p)) / e**2
n

In [ ]:
# Sample 
# 92 students
ssd = 23.7
n = 92
mean = 145.2

sample_error = ssd / n**0.5
sample_error

In [ ]:
# Sample error * z value
mean, sample_error * 1.96

# Hypothesis testing

- The null hypothesis
    1. We are working with two samples of data and want to assess if they are statistically different
    2. Null hypothesis states that they are the same, and any differences are due to chance
    3. By selecting the right test to the type of data (proportion, mean) and assumptions, we can derive standardised scores such as chi squared score or T score which can be looked up to determine the probability value (p value)
    4. The probability of observing the difference if the null hypothesis is true (or more extreme difference)
    5. When p values less than 0.05 we usually reject, but we should be aware of multiple testing
- Power analysis
    - Definition
        - Beta is the failure to reject a false null hypothesis
        - Probability of missing an effect that is really there
        - Power probability of detecting an effect that is really there
        - Power = 1 - b
        - By convention, we want 80% power
    - Calculations
        - Minimum practical effect
            - From business, or former studies
        - Standard deviation
            - The control likely has been measured before
        - One or two tailed test    
    - Factors
        - Sample size (more good)
        - Variability (more bad)
        - Difference between mean and hypothetical mean (more better)
        - Significance level required (more bad)
        - One tail vs two tailed (one better)
    - Rough calculation
        - n ~= (16 * σ ** 2) / d ** 2
        - Where σ is SD, d is difference in practice

# Comparing two populations

- Comparing success rate (proportion) with z score
- Comparing means of two populations 
    - With z score (large sample)
    - With T score (small sample)
        - Equal variance or unequal variance (more conservative)
        - One tailed (hypothesis that it will be bigger/smaller) or two tailed (no hypothesis, more conservative)    
- Paired comparisons

In [ ]:
# T Tests
# ttest_ind: T-test from two pandas Series input

results = stats.ttest_ind(data.loc[data['survived']==1, 'fare'], data.loc[data['survived']==0, 'fare'])
logging.info("Relationship between survival and fare paid")
logging.info(f"T-test t-stat: {results.statistic:.3f}, p-value: {results.pvalue:.3f}. (expected difference)")

logging.info("")

results = stats.ttest_ind(data.loc[data['name_u']==1, 'age'], data.loc[data['name_u']==0, 'age'])
logging.info("Relationship between letter 'u' in name and age")
logging.info(f"T-test t-stat: {results.statistic:.3f}, p-value: {results.pvalue:.3f}. (expected no difference)")

In [ ]:
# Chi squared tests
# chi2_contingency: Chi squared tests from crosstab input
results = stats.chi2_contingency(pd.crosstab(data['survived'], data['sex']))
logging.info("Relationship between survival and sex")
logging.info(f"Chi-squared Chi-value: {results[0]:.3f}, p-value: {results[1]:.3f}. (expected difference)")

logging.info("")

results = stats.chi2_contingency(pd.crosstab(data['name_u'], data['sex']))
logging.info("Relationship between letter 'u' in name and sex")
logging.info(f"Chi-squared Chi-value: {results[0]:.3f}, p-value: {results[1]:.3f}. (expected no difference)")

In [ ]:
# Paired tests
# Rather than getting mean of group A and group B... 
# both groups receive treatment at different times
# The difference between their experiences in A, B is C = A-B 
# C has it's own mean and SD and much more statistical power
mean = -0.6
sd = 0.61
n = 10
t_critical_value = 2.26

population_mean_lower = mean - t_critical_value * (sd / n**0.5)
population_mean_upper = mean + t_critical_value * (sd / n**0.5)
print(f"{population_mean_lower:.3f}, {population_mean_upper:.3f}")

# As confidence interval does not cross 0, this indicates significance

# Regression

- Least squares regression
- Statsmodel regression

- Linear regression
    - Key assumptions
        - Linearity 
        - Independence of data
        - Normally distributed data
        - Homoscedacity - errors are evenly distributed
    - Problems
        - Heteroscedacity
        - Normality Q-Q plots
        - Outliers
        - Multicollinearity
        

- Logistic regression
  - Logistic Function
    - Sigmoid function, from infinity to 0, 1 (y)
    - Logit function, from 0, 1 to infinities (y)
  - Maximum likelihood estimation - Used to estimate betas
  - Log likelihood function
  - Gradient decent to maximise likelihood
  - Estimating B: start random, compute gradients and update betas, repeat until converge
  - Complexity analysis O(MNI), Each datapoint, each feature, each iteration
  - Mini-batch gradient decent (best for large data)
  - Further resources
    - Data Science Pro https://www.youtube.com/watch?v=gN79XvB7vTo&list=PLY1Fi4XflWSsLoOQr-Ee2R4qRFejtCFRh&index=2
    - Brandon Foltz https://www.youtube.com/watch?v=NmjT1_nClzg
- Regularisation, ridge regression vs lasso
    - L1 lasso - absolute value of coefficients
    - L2 ridge - squared value of coefficients
    - Can combine for elastic net
    - L1 can zero out coefficients, useful for feature selection


In [ ]:
# Least squares regression
values = [(95, 85),
        (85, 95),
        (80, 70),
        (70, 65),
        (60, 70)]

x = [z[0] for z in values]
y = [z[1] for z in values]

mean_x = sum(x) / len(x)
mean_y = sum(y) / len(y)

x_diff = [xn - mean_x for xn in x]
y_diff = [yn - mean_y for yn in y]

sd_x = (sum([xn**2 for xn in x_diff]) / len(x))**0.5
sd_y = (sum([yn**2 for yn in y_diff]) / len(y))**0.5

cov = 0
for i in range(len(x_diff)):
    cov += x_diff[i] * y_diff[i] 

correlation = cov / (len(x) * sd_x * sd_y)

print(round(correlation, 3))

b = correlation * (sd_y / sd_x)
print(b)

a = mean_y - b * mean_x
print(a)

round(a + b*80, 3)

statsmodels.api

- Includes param p values unlike sklearn
- API different; data -> model then fit, rather than sklearn model then data -> fit

In [ ]:
# Statsmodel linear regression
target = "survived"
features = ["pclass", "age"]
data_ready = data.dropna(subset=['age', "pclass", "survived"])

data_ready["_constant"] = 1
features.append("_constant")

# Need to add constant either through data
# Beware Int64 types
# Predictions are through 'results' variable
model = sm.OLS(data_ready[target], data_ready[features])
results = model.fit()

logging.info(results.pvalues)
logging.info(results.params)
logging.info(results.rsquared)

results.predict(data_ready[features])

In [ ]:
# TODO add full linear and logistic regression

In [ ]:
# https://github.com/tatwan/Linear-Regression-Implementation-in-Python/blob/master/Linear_Regression_Python.ipynb

In [ ]:
def computeCost_m(x, y, theta):
    m = len(y)
    h_x = np.dot(x, theta)
    j = np.sum(np.square(h_x - y))/(2*m)
    return j

In [ ]:
theta_init

In [ ]:
theta_init = np.zeros((len(features), 1))
computeCost_m(data_ready[features], data_ready[target], theta_init)

In [ ]:
def gradientDescentMulti(X, Y, theta, alpha, num_iters):
    m = len(Y)
    p = np.copy(X)
    t = np.copy(theta)
    j = []
    print('Running Gradient Descent')
    for i in range(0,num_iters+1):
        cost = computeCost_m(p, Y, t)
        j.append(cost)
        h_x = np.dot(p, t)
        err = h_x - Y
        for f in range(theta.size):
            t[f] = t[f] - alpha/m *(np.sum((np.dot(p[:,f].T, err))))
    return j, t

In [ ]:
# theta_init = np.zeros((3, 1))
alpha = 0.01
num_iters = 5000
theta_init = np.zeros((19, 1))
cost, theta_final = gradientDescentMulti(x_norm, Y, theta_init, alpha, num_iters)

plt.figure()
plt.plot(cost)
plt.xlabel('No. of Iterations')
plt.ylabel('Cost')
plt.show()

# Correlations
- Pearson correlation coefficient
- Spearman rank correlation

In [ ]:
x_raw = "10 9.8 8 7.8 7.7 1.7 6 5 1.4 2 "
y_raw = "200 44 32 24 22 17 15 12 8 4"

x = [float(xn) for xn in x_raw.strip().split(" ")]
y = [float(yn) for yn in y_raw.strip().split(" ")]

In [ ]:
# Pearson correlation coefficient
mean_x = sum(x) / len(x)
mean_y = sum(y) / len(y)

x_diff = [xn - mean_x for xn in x]
y_diff = [yn - mean_y for yn in y]

sd_x = (sum([xn**2 for xn in x_diff]) / len(x))**0.5
sd_y = (sum([yn**2 for yn in y_diff]) / len(y))**0.5

cov = 0
for i in range(len(x_diff)):
    cov += x_diff[i] * y_diff[i] 

correlation = cov / (len(x) * sd_x * sd_y)

print(round(correlation, 3))

In [ ]:
# Spearman rank coefficient
x_rank = dict([(x_rank, i) for i, x_rank in enumerate(sorted(x))])
y_rank = dict([(y_rank, i) for i, y_rank in enumerate(sorted(y))])

x = [(xn, x_rank[xn]) for xn in x]
y = [(yn, y_rank[yn]) for yn in y]

d = 0
for i in range(len(x)):
    d += (x[i][1] - y[i][1])**2

spearman = 1 - ((6 * d) / (len(x) * (len(x)**2 - 1)))    
print(round(spearman, 3))

# Time series analysis

Linear approaches
- ARIMA
    - Efficient and interpretable
    - Strong model assumptions, simple

State-space models
- Kalman filter, Hidden Markov model
    - More flexible and interpretable
    - Expensive inference

Deep sequence models (DCRNN)
- RNN, attention
    - Efficient and non-linear
    - Hard to interpret
- DCRNN
    - K-step diffusion
    - Unit: DCGRU Diffusional convolutional gated recurrent unit
        - input: sequence of history graphs
        - outputs: sequence of future graphs

Historical average (google) is a really good baseline
ARIMA can/can’t be better, good in short term, not so good in longer time
DCRNN best in short term and long term 
